In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr

In [ ]:
from crims2s.dataset import S2SDataset, TransformedDataset
from crims2s.transform import normalize_dataset

In [ ]:
ECMWF_LSM = "***BASEDIR***/training-input/ecmwf-hindcast-lsm-20200102.nc"

In [ ]:
GEOGRAPHY_FILE = "***BASEDIR***/from-s2s/ecmwf-orog.nc"

In [ ]:
DATASET_DIR = "***BASEDIR***/mlready/2021-09-14-lean/"

In [ ]:
lsm = xr.open_dataset(ECMWF_LSM)

In [ ]:
lsm

In [ ]:
def fix_s2s_dataset_dims(s2s_dataset):
    """Fix the dims of a file coming directly from the S2S data repository."""
    return s2s_dataset.rename(X='longitude', Y='latitude', L='lead_time')

In [ ]:
geo = fix_s2s_dataset_dims(xr.open_dataset(GEOGRAPHY_FILE))

In [ ]:
geo

In [ ]:
(geo.isel(lead_time=slice(1, None)) - geo.isel(lead_time=slice(0, -2))).sum(dim=['longitude', 'latitude']).orog.plot()

In [ ]:
lsm.longitude == geo.longitude

In [ ]:
geo.isel(lead_time=0).orog.plot()

In [ ]:
geo.isel(lead_time=0).slt.plot()

In [ ]:
geo.mean()

In [ ]:
geo.std()

In [ ]:
new_feature_dataset = normalize_dataset(geo[['orog']]).isel(lead_time=0).to_array().to_dataset(name="features")

In [ ]:
new_feature_dataset

In [ ]:
class AddGeographyFeatures:
    def __init__(self, geography_file):
        geo_dataset = fix_s2s_dataset_dims(xr.open_dataset(geography_file))
        subset = geo_dataset[['orog']]
        
        geo = normalize_dataset(subset)
        self.geo_features = geo.to_array().to_dataset(name="features")
        
    def __call__(self, batch):
        features = batch['features']        
        new_features_dataset = xr.concat([features, self.geo_features], dim='variable')
        
        batch['features'] = new_features_dataset
        
        return batch

In [ ]:
t = AddGeographyFeatures(GEOGRAPHY_FILE)

In [ ]:
d = TransformedDataset(S2SDataset(DATASET_DIR), t)

In [ ]:
d[0]['features'].features.sel(variable='orog').plot()

In [ ]:
feature_dataset = d[0]['features']

In [ ]:
merged_features = xr.concat([feature_dataset, new_feature_dataset], dim='variable')

In [ ]:
merged_features

In [ ]:
merged_features.features.sel(variable='orog').isel(lead_time=0).plot()